In [1]:
!pip install adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.5 MB 15.4 MB/s 
     |████████████████████████████████| 895 kB 61.7 MB/s 
     |████████████████████████████████| 50 kB 7.5 MB/s 
     |████████████████████████████████| 3.3 MB 58.2 MB/s 
     |████████████████████████████████| 264 kB 14.4 MB/s 
     |████████████████████████████████| 243 kB 63.5 MB/s 
     |████████████████████████████████| 119 kB 27.0 MB/s 


In [2]:
cd drive/MyDrive/CAMemBERT2/

/content/drive/MyDrive/CAMemBERT2


In [6]:
from transformers import AutoModelWithHeads,AutoConfig,Trainer,TrainingArguments,EarlyStoppingCallback
import numpy as np
import pdb
from sklearn.metrics import cohen_kappa_score,f1_score,accuracy_score,fbeta_score
from scipy.stats import spearmanr
from trainer_utils import get_dataset
import torch
import pandas as pd
from transformers.adapters.composition import Fuse

def normalise_scores(example):
    score = example['labels']/40
    return {'labels':score}

dataset_obj,dataset_dict = get_dataset(['aes'],max_length=512)
# prepare for model
dataset_dict = dataset_dict.remove_columns(['dataset','labels']).rename_column('scores','labels')
dataset_dict = dataset_dict.map(normalise_scores)

/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])


KeyboardInterrupt: ignored

In [20]:
config_ = AutoConfig.from_pretrained('distilroberta-base',num_labels=1)
model_ = AutoModelWithHeads.from_pretrained('distilroberta-base',config=config_)
model_.load_adapter('/content/drive/MyDrive/CAMemBERT2/adapters/ged', load_as="ged", with_head=False)
model_.load_adapter('/content/drive/MyDrive/CAMemBERT2/adapters/ner', load_as="ner", with_head=False)
adapter_setup = Fuse(['ged','ner'])
adapter_setup_ = Fuse('ged')
model_.add_fusion(adapter_setup_)
model_.add_classification_head("aes_fuse", num_labels=1,overwrite_ok=True)
model_.set_active_adapters(adapter_setup_)
model_.train_fusion(adapter_setup_)
lr,bs = 6e-5,8

def compute_metrics(p):
    logits, labels = p.predictions,p.label_ids
    preds,labels = np.rint(logits.flatten()*40),np.rint(labels*40)
    metrics_dic = {
              "rmse": np.sqrt(np.mean((preds-labels)**2)),
              "pearson": np.corrcoef(preds,labels)[0,1],
              "spearman" : spearmanr(preds, labels)[0],
              "kappa":cohen_kappa_score(np.rint(preds),np.rint(labels),weights='quadratic')
              }
    return metrics_dic

label_names = ['labels']
args = TrainingArguments(
        output_dir = '/content',
        save_total_limit = 0,
        evaluation_strategy = "epoch",
        learning_rate = lr,
        per_device_train_batch_size=bs,
        per_device_eval_batch_size=bs,
        remove_unused_columns=False,
        num_train_epochs = 20,
        weight_decay = 0.01,
        save_strategy = 'epoch',
        load_best_model_at_end=True,
        metric_for_best_model='pearson',
        label_names=label_names
    )


trainer = Trainer(
    model=model_,
    args=args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['dev'],
    compute_metrics=compute_metrics
    )

early_stopping = EarlyStoppingCallback(early_stopping_patience = 5)
trainer.add_callback(early_stopping)
trainer.train()
trainer.evaluate(dataset_dict['test'])
pd.DataFrame(trainer.state.log_history).to_csv(f'results/aesadapters_fuse_ged_distilroberta_lr{lr}_bs{bs}_norm.csv')


loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_

Epoch,Training Loss,Validation Loss,Rmse,Pearson,Spearman,Kappa
1,No log,0.013147,4.651119,0.619065,0.630534,0.492689
2,No log,0.014810,4.820841,0.639623,0.624849,0.533269
3,No log,0.018790,5.494531,0.646463,0.627610,0.526059
4,0.023800,0.018104,5.396905,0.656705,0.637492,0.534276
5,0.023800,0.023780,6.201266,0.625230,0.604602,0.384871
6,0.023800,0.021436,5.898938,0.669907,0.651771,0.442106
7,0.023800,0.038342,7.849857,0.661384,0.643753,0.317533
8,0.016700,0.015426,4.961880,0.685377,0.658091,0.540748
9,0.016700,0.016217,5.041599,0.671035,0.646096,0.567777
10,0.016700,0.014717,4.840494,0.685364,0.663145,0.552265


***** Running Evaluation *****
  Num examples = 79
  Batch size = 8
Saving model checkpoint to /content/checkpoint-133
Configuration saved in /content/checkpoint-133/ged/adapter_config.json
Module weights saved in /content/checkpoint-133/ged/pytorch_adapter.bin
Configuration saved in /content/checkpoint-133/ner/adapter_config.json
Module weights saved in /content/checkpoint-133/ner/pytorch_adapter.bin
Configuration saved in /content/checkpoint-133/ged/adapter_fusion_config.json
Module weights saved in /content/checkpoint-133/ged/pytorch_model_adapter_fusion.bin
***** Running Evaluation *****
  Num examples = 79
  Batch size = 8
Saving model checkpoint to /content/checkpoint-266
Configuration saved in /content/checkpoint-266/ged/adapter_config.json
Module weights saved in /content/checkpoint-266/ged/pytorch_adapter.bin
Configuration saved in /content/checkpoint-266/ner/adapter_config.json
Module weights saved in /content/checkpoint-266/ner/pytorch_adapter.bin
Configuration saved in /con

In [13]:
adapter_setup

Fuse[Stack[ged, ner]]